In [2]:
# random / nonduplication / randomforest
### 개별 정확도 34% / 전체 정확도 8.4%

import numpy as np
import pandas as pd

# 정확도 측정용

test_data_mul_value = pd.read_csv("./busan/output(LE).csv")


def clean_and_convert(value):
    value = value.strip("[]").replace(",", "")  # 대괄호와 쉼표 제거
    value_list = value.split()  # 공백을 기준으로 분리
    return [int(item) for item in value_list]  # 각 값을 정수로 변환

test_data_mul_value['start'] = test_data_mul_value['start'].apply(clean_and_convert)
test_data_mul_value['main'] = test_data_mul_value['main'].apply(clean_and_convert)
test_data_mul_value['end'] = test_data_mul_value['end'].apply(clean_and_convert)
test_data_mul_value = test_data_mul_value.iloc[:,7:]

data = pd.read_csv("./busan/output(random_duplication).csv")

combined_data = pd.concat([data, test_data_mul_value], axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(combined_data.iloc[:, :7], combined_data.iloc[:,7:], test_size=0.2, random_state=42)

y_test_val = y_test.iloc[:,3:]

y_train = y_train.iloc[:,:3]
y_test = y_test.iloc[:,:3]


In [6]:

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 스케일러 추가
    ('rf', RandomForestClassifier(random_state=42))  # 랜덤포레스트 분류기
])

param_grid = {
    'rf__n_estimators': [100],
    'rf__max_depth': [4, 6, 8],
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, verbose=2, cv=5, n_jobs=4)   

grid_search.fit(X_train, y_train)



Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

In [ ]:
y_pred = pipeline.predict(X_test)

count = 0
for i in range(len(y_pred)):
  if y_pred[i,0] in y_test_val.iloc[i,0]:
    count +=1
  if y_pred[i,1] in y_test_val.iloc[i,1]:
    count +=1
  if y_pred[i,2] in y_test_val.iloc[i,2]:
    count +=1

print(count/len(y_pred)/3)

count = 0
for i in range(len(y_pred)):
  if (y_pred[i,0] in y_test_val.iloc[i,0]) and (y_pred[i,1] in y_test_val.iloc[i,1]) and (y_pred[i,2] in y_test_val.iloc[i,2]):
    count +=1

print(count/len(y_pred))